In [ ]:
import os
import pywt
#from wavelets.wave_python.waveletFunctions import *
import itertools
import numpy as np
import pandas as pd
from scipy.fftpack import fft
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

In [ ]:
def upwell(ektrxs, ektrys, coast_angle):
    pi = 3.141592653
    degtorad = pi/180.
    alpha = (360 - coast_angle) * degtorad
    s1 = np.cos(alpha)
    t1 = np.sin(alpha)
    s2 = -1 * t1
    t2 = s1
    res = []
    for x, y in zip(ektrxs, ektrys):
        x = float(x)
        y = float(y)
        perp = (s1 * x) + (t1 * y)
        para = (s2 * x) + (t2 * y)
        res.append(perp/10)
    return(res)
df_30_whole = pd.read_csv('1982-22-30.csv')
df_30_whole = df_30_whole.iloc[1: , :]
df_30_whole['combined'] = upwell(df_30_whole['ektrx'], df_30_whole['ektry'], 210)
df_30_whole

In [ ]:
N = df_30_whole.shape[0]
t0=1982
dt= (1/12)
time = np.arange(0, N) * dt + t0
signal = df_30_whole['combined'].values
plt.figure(figsize=(15, 10))
plt.plot(time, signal)
plt.title('Signal', fontsize=18)
plt.ylabel('value', fontsize=18)
plt.xlabel('year', fontsize=18)
plt.show()

In [ ]:
def get_fft_values(y_values, T, N, f_s):
    N2 = 2 ** (int(np.log2(N)) + 1) # round up to next highest power of 2
    f_values = np.linspace(0.0, 1.0/(2.0*T), N2//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N2 * np.abs(fft_values_[0:N2//2])
    return f_values, fft_values

# def plot_fft_plus_power(ax, time, signal, plot_direction='horizontal', yticks=None, ylim=None):
#     dt = time[1] - time[0]
#     N = len(signal)
#     fs = 1/dt
#     print(dt,fs)
#     variance = np.std(signal)**2
#     f_values, fft_values = get_fft_values(signal, dt, N, fs)
#     print(fft_values)
#     fft_power = variance * abs(fft_values) ** 2
#     if plot_direction == 'horizontal':
#         ax.plot(f_values, fft_values, 'r-', label='Fourier Transform')
#         ax.plot(f_values, fft_power, 'k--', linewidth=1, label='FFT Power Spectrum')
#     elif plot_direction == 'vertical':
#         scales = 1./f_values
#         scales_log = np.log2(scales)
#         ax.plot(fft_values, scales_log, 'r-', label='Fourier Transform')
#         ax.plot(fft_power, scales_log, 'k--', linewidth=1, label='FFT Power Spectrum')
#         ax.set_yticks(np.log2(yticks))
#         ax.set_yticklabels(yticks)
#         ax.invert_yaxis()
#         ax.set_ylim(20, -1)
#     ax.legend()

# fig, ax = plt.subplots(figsize=(12,3))
# ax.set_xlabel('Frequency [Hz / year]', fontsize=18)
# ax.set_ylabel('Amplitude', fontsize=18)
# plot_fft_plus_power(ax, time, signal)
# plt.show()
dt = time[1] - time[0]
N = len(signal)
fs = 1/dt
variance = np.std(signal)**2
f_values, fft_values = get_fft_values(signal, dt, N, fs)
plt.plot(f_values, fft_values, 'r-', label='Fourier Transform')



In [ ]:
n = len(time)
dt = 1/12
fhat = np.fft.fft(signal, n) #computes the fft
psd = fhat * np.conj(fhat)/n
freq = (1/(dt*n)) * np.arange(n) #frequency array
idxs_half = np.arange(1, np.floor(n/2), dtype=np.int32) #first half index
fig, ax = plt.subplots(2,1)
ax[0].plot(time, signal, color='b', lw=0.5, label='Noisy Signal')
ax[1].plot(freq[idxs_half], np.abs(psd[idxs_half]), color='b', lw=0.5, label='PSD noisy')
ax[1].set_xlabel('Frequencies in Hz')
ax[1].set_ylabel('Amplitude')
ax[1].legend()

In [ ]:
import scaleogram as scg 
import pywt

# choose default wavelet function for the entire notebook
scg.set_default_wavelet('cmor1-1.5')

In [ ]:
scales = scg.periods2scales(np.arange(1.0, 20))
scg.cws(signal, scales=scales, figsize=(6.9,2.3)); 

In [ ]:
scales = scg.periods2scales(np.arange(1.0, 20))
scg.cws(signal_24, scales=scales, figsize=(6.9,2.3)); 